In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2016,22)

In [2]:
input_text="""Filesystem            Size  Used  Avail  Use%
/dev/grid/node-x0-y0   10T    8T     2T   80%
/dev/grid/node-x0-y1   11T    6T     5T   54%
/dev/grid/node-x0-y2   32T   28T     4T   87%
/dev/grid/node-x1-y0    9T    7T     2T   77%
/dev/grid/node-x1-y1    8T    0T     8T    0%
/dev/grid/node-x1-y2   11T    7T     4T   63%
/dev/grid/node-x2-y0   10T    6T     4T   60%
/dev/grid/node-x2-y1    9T    8T     1T   88%
/dev/grid/node-x2-y2    9T    6T     3T   66%
"""
input_lines = input_text.split('\n')

In [3]:
import time
import re
import sys
from heapq import heappush, heappop
from copy import deepcopy

# regex for matching grid data
pattern_df = re.compile(r'.*x(\d+)-y(\d+)\s+(\d+)T\s+(\d+)T\s+(\d+)T\s+(\d+)%')
SIZE=0
USED=1
AVAIL=2
PERCENT=3

class HeapEntry:
    """
    Class heap entry, gives a sortable object for the heap, 
    workaround for heapq not liking dict
    """
    def __init__(self, my_steps, my_node, my_data):
        self.steps = my_steps
        self.target = (0,0)
        self.node = my_node
        self.data = my_data

    def __lt__(self, other):
        return (self.steps, self.node) < (other.steps, other.node)

    def __str__(self):
        minimum, maximum = get_range(self.data)
        #retval = f"Steps: {self.steps}, Position: ({self.node})\n"
        retval = ""
        for y_val in range(0,maximum[1]+1):
            for x_val in range(0,maximum[0]+1):
                symbols = '  '
                pos = (x_val, y_val)
                if pos == self.target:
                    symbols = '()'
                if pos == self.node:
                    symbols = '[]'
                retval += f"{symbols[0]} {self.data[pos][USED]}T/{self.data[pos][SIZE]}T {symbols[1]}"
                if x_val < maximum[0]:
                    retval += " -- "
            retval += "\n"
            if y_val < maximum[1]:
                for x_val in range(0,maximum[0]+1):
                    retval += "    |    "
                    if x_val < maximum[0]:
                        retval += "    "
            retval += "\n"
        return retval

def parse_input(lines):
    """
    Function to parse input
    """
    # not sure what the best data structure is yet, so starting with dict of 
    #dicts
    # changed my mind already, dict of dict with position tuple as key is where I
    # want to start
    # changed to dict of tuples with position tuple as key
    nodes = {}
    # walk linies
    for line in lines:
        # check if this is a node line
        match = pattern_df.match(line)
        if match:
            # get data from regex match
            # Filesystem              Size  Used  Avail  Use%
            # /dev/grid/node-x0-y0     85T   64T    21T   75%
            position = [0, 0]
            position[0], position[1], size, used, avail, percent = match.groups()
            # useless operation to make pylint happy later
            # I don't think we are actually using percent, so dropping it from data
            # structure.  for that matter, we provbably aren't using size either
            percent = int(percent)
            # build position[0]/position[1]
            nodes[(int(position[0]),int(position[1]))] = tuple(
                [int(size), int(used), int(avail)]
            )
    # return data
    return nodes

def get_range(nodes):
    """
    Function to get range of x/y values
    Not needed for part 1, anticipating part 2
    """
    if not nodes:
        return ({0: 0, 1: 0}, {0: 0, 1: 0})

    x_values, y_values = zip(*nodes)
    
    return (
        {0: min(x_values), 1: min(y_values)},
        {0: max(x_values), 1: max(y_values)}
    )
  
def get_neighbors(nodes, position):
    """
    Function to get neighbors of a node
    It doesn't looke like I need this for part 1
    """
    (minimum, maximum) = get_range(nodes)
    found_neighbors = set()
    if position[1] > minimum[1]:
        found_neighbors.add((position[0], position[1]-1))
    if position[1] <  maximum[1]:
        found_neighbors.add((position[0], position[1]+1))
    if position[0] > minimum[0]:
        found_neighbors.add((position[0]-1, position[1]))
    if position[0] <  maximum[0]:
        found_neighbors.add((position[0]+1, position[1]))
    return found_neighbors

def can_move(nodes, node_a, node_b):
    """
    Function to determine if data can move from node_a to node_b
    """
    # {'size': '85', 'used': '64', 'avail': '21', 'percent': '75'}
    #Nodes A and B are not the same node.
    if node_a == node_b:
        return False
    #Node A is not empty (its Used is not zero).
    if nodes[node_a][USED] == 0:
        return False
    #The data on node A (its Used) would fit on node B (its Avail).
    # note to self <= and >= don't mean the same thing
    if nodes[node_b][AVAIL] >= nodes[node_a][USED]:
        return True
    return False
    
def move_data(nodes, node_a, node_b):
    """
    function to move data from node_a to node_b
    """
    #clone nodes
    tmp_nodes = deepcopy(nodes)
    # get lists of node data to manipulate
    node_a_data = list(tmp_nodes[node_a])
    node_b_data = list(tmp_nodes[node_b])
    # add node_a data to node_b
    node_b_data[USED] += node_a_data[USED]
    # relalculate available
    node_b_data[AVAIL] = node_b_data[SIZE] - node_b_data[USED]
    # set node_a data to empty
    node_a_data[USED] = 0
    # recalculate available
    node_a_data[AVAIL] = node_a_data[SIZE] - node_a_data[USED]
    # convert back to tuples
    tmp_nodes[node_a] = tuple(node_a_data)
    tmp_nodes[node_b] = tuple(node_b_data)
    # return new structure
    return tmp_nodes

def find_empty(nodes):
    """
    Function to find the empty data node
    """
    # walk nodes
    for node, data in nodes.items():
        # find empty
        if data[USED] == 0:
            return node
            
def find_shortest_path(nodes, start_node, target_node):
    """
    Function to find shortest path
    """
    # init heap
    heap = []
    heappush(heap,HeapEntry(0, start_node, nodes))
    min_steps = float('infinity')
    already_seen = set()
    # init sentinel
    sentinel = 0
    # process heap
    while heap:
        # increment sentinel
        sentinel += 1
        # check sentinel
        #if sentinel > 100000:
        #    print("Breaking loop")
        #    break
        # get next test case
        #steps, current_node, nodes = heappop(heap)
        # noe a HeapEntry object
        current = heappop(heap)
        #print(current)
        # lets not test longer paths
        if current.steps > min_steps:
            continue
        # solved?
        if current.node == target_node:
            if current.steps < min_steps:
                min_steps = current.steps
            continue
        if str(current) in already_seen:
            continue
        already_seen.add(str(current))
        # for each node in current map
        for node in current.data:
            # for each neighbor
            for neighbor in get_neighbors(current.data, node):
                # can we move to neighbor?
                if can_move(current.data, node, neighbor):
                    if node == current.node:
                        # set current to neighbor, and add to heap
                        heappush(heap,HeapEntry(current.steps+1, neighbor, move_data(current.data, node, neighbor)))
                    else:
                        # add new map to heap
                        heappush(heap,HeapEntry(current.steps+1, current.node, move_data(current.data, node, neighbor)))
                        
    return min_steps

In [7]:
my_nodes = parse_input(input_lines)
target_node = (0,0)
source_node = (get_range(my_nodes)[1][0],0)
current = HeapEntry(0,source_node, my_nodes)
print(current, find_empty(current.data))
exit()

print(f"Moving from {source_node} to {target_node}")
my_steps = find_shortest_path(my_nodes, source_node, target_node)
print(my_steps)

( 8T/10T ) --   7T/9T   -- [ 6T/10T ]
    |            |            |    
  6T/11T   --   0T/8T   --   8T/9T  
    |            |            |    
  28T/32T   --   7T/11T   --   6T/9T  

 (1, 1)
Moving from (2, 0) to (0, 0)
7


# str(my_nodes)

In [57]:
def min_steps_remaining(floors):
    score = 0
    for idx, floor in enumerate(floors):
        score += (3 - idx) * len(floor) 
    return score // 2
    
def h_score(floors, stops, threshold):
    score = min_steps_remaining(floors)
    print(f"min_steps_remaining: {score}")
    # if it will take more elevator stops to complete than we have left
    # before min_solved, then we aren't on the right track, so lets bump
    # score up to deprioritize this route
    if stops // 2 + score > threshold:
        score *= 1000
    else:
        score *= 5
    
    if len(floors[0]) == 0:
        score -= 1000
        if len(floors[1]) == 0:
            score -= 2000
            if len(floors[3]) > len(floors[2]):
                score -= 3000
    return score

In [82]:
h_score((('PoG',), ('CoM',), ('CoG', 'RuG', 'TmG', 'PmG'), ('PoM', 'RuM', 'PmM', 'TmM')), 31, float('infinity'))

min_steps_remaining: 4


20

In [114]:
my_set = set()
for my_str in ['RuG', 'PmG', 'CoG', 'TmG']:
    my_set.add(my_str)
print(my_set)
sorted(list(my_set))

{'CoG', 'PmG', 'TmG', 'RuG'}


['CoG', 'PmG', 'RuG', 'TmG']